In [1]:
import random
import pickle
import json
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words


def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)


def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['response'])
            break
    return result


print("GO! Bot is running!")
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\talas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\talas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\talas\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


GO! Bot is running!
hi
1/1 [==============================] - 0s 81ms/step
Hey
hello
1/1 [==============================] - 0s 12ms/step
Hello!
tell me about phoenix group
1/1 [==============================] - 0s 19ms/step
Founded by Mr. Amit Mahajan in 2016, The Phoenix Group is a conglomerate of four companies â€“ Phoenix Innovations LLC,         Griffyn Robotech, Xcaliber Infotech, and Bit Mapper Integration Technologies.
can you give me an idea about xcaliber
1/1 [==============================] - 0s 11ms/step
Xcaliber is a software service company, It helps client upgrade through innovations and technology.
what do you know about griffyn robotech
1/1 [==============================] - 0s 6ms/step
Griffyn Robotech deals in Robotech and Automatioin, It deals in building a complete set of A.I powered tools for the                        Industry 4.0
bye
1/1 [==============================] - 0s 20ms/step
Sad to see you go. Bye


KeyboardInterrupt: Interrupted by user